In [ ]:
import pandas as pd
from core_util.proxy import enable_proxy
from tqdm import tqdm

enable_proxy()

In [22]:
df_path = '/home/dell/code/ercot/GIS_Report_January2026.xlsx'
df = pd.read_excel(df_path,sheet_name=4,skiprows=34)
# Excel rows 31–35 → pandas rows 30–34
hdr = pd.read_excel(
    df_path,
    header=None,
    skiprows=30,
    nrows=5,sheet_name=4
)

# emulate merged cells
# hdr = hdr.ffill(axis=1).ffill(axis=0)

headers = []
for col in hdr.columns:
    vals = (
        hdr[col]
        .dropna()
        .astype(str)
        .str.strip()
    )

    # keep unique in original order
    unique_vals = pd.Series(vals).drop_duplicates().tolist()

    headers.append(" ".join(unique_vals))
df.columns = headers



In [23]:
headers

['INR',
 'Project Name',
 'GIM Study Phase',
 'Interconnecting Entity',
 'POI Location',
 'County',
 'CDR Reporting Zone',
 'Projected COD',
 'Fuel',
 'Technology',
 'Capacity (MW)',
 'Change indicators: Proj Name, MW Size, COD, SFS/NtP, FIS Request, Status Change INA-to-PLN',
 'Approval Date for Submission of Proof of Site Control',
 'Screening Study Started',
 'Screening Study Complete',
 'FIS Requested',
 'FIS Approved',
 'Economic Study Required',
 'IA Signed',
 'Financial Security and Notice to Proceed Provided',
 'Air Permit',
 'GHG Permit',
 'Water Availability',
 'Meets Planning Guide Section 6.9(1) Requirements for Inclusion in Planning Models',
 'Meets All Planning Guide Section 6.9 Requirements for Inclusion in Planning Models',
 'Meets Planning Guide QSA (Section 5.9) Prerequisites',
 'Construction Start',
 'Construction End',
 'Approved for Energization',
 'Approved for Synchronization',
 'Comment']

In [24]:
in_play = df[~df['Approved for Synchronization'].isna()]
in_construction = df[df['Approved for Synchronization'].isna()]

In [25]:
in_play[['POI Location','CDR Reporting Zone','Fuel','Capacity (MW)']]

,POI Location,CDR Reporting Zone,Fuel,Capacity (MW)
0,59903 Bearkat 345kV,WEST,WIN,162.10
2,tap 345kV 1444 Brown – 3422 Killeen,NORTH,WIN,302.40
3,tap 345kV 76015 Cedar Canyon - 7053 Noelke,WEST,WIN,265.40
5,11305 Dermott 345kV,WEST,WIN,242.60
7,6216 Bluff Creek 138kV,WEST,WIN,44.00
...,...,...,...,...
856,60501 Tesla 345 kV,PANHANDLE,SOL,166.57
857,44880 Waterh_POI_5 345kV,SOUTH,SOL,105.98
1002,"Bus Name: TnMainland1,\nBus Number: 38830,\n\n",HOUSTON,OTH,207.30
1150,Tap 345kV 11010 Wolf - 1018 Moss,WEST,SOL,150.71


In [26]:
from geopy.geocoders import Nominatim
import geopy
geopy.geocoders.options.default_user_agent = 'my_app/1'
geopy.geocoders.options.default_timeout = 7

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Create once
geolocator = Nominatim(user_agent="my_real_app_name_here")
geocode = RateLimiter(
    geolocator.geocode,
    min_delay_seconds=1.1,  # MUST be >1 sec
    max_retries=3,
    error_wait_seconds=5
)

def address_to_coordinates(address):
    location = geocode(address)
    if location:
        return location.latitude, location.longitude
    return None


In [29]:
coordinates = []
for address in tqdm(in_play['POI Location']):
    coord = address_to_coordinates(address)
    coordinates.append(coord)

100%|██████████| 117/117 [08:25<00:00,  4.32s/it]


In [30]:
in_play["coordinates"] = coordinates


/tmp/ipykernel_1364549/1664746589.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_play["coordinates"] = coordinates


In [31]:
import pickle
pickle.dump(in_play,open('./nodes_in_play_with_coords_2026.1.pkl','wb'))

In [32]:
in_play

,INR,Project Name,GIM Study Phase,Interconnecting Entity,POI Location,County,CDR Reporting Zone,Projected COD,Fuel,Technology,...,Water Availability,Meets Planning Guide Section 6.9(1) Requirements for Inclusion in Planning Models,Meets All Planning Guide Section 6.9 Requirements for Inclusion in Planning Models,Meets Planning Guide QSA (Section 5.9) Prerequisites,Construction Start,Construction End,Approved for Energization,Approved for Synchronization,Comment,coordinates
0,15INR0064b,Harald (BearKat Wind B),"SS Completed, FIS Completed, IA","McCrae Wind Energy II, LLC",59903 Bearkat 345kV,Glasscock,WEST,2026-02-28,WIN,WT,...,Not Required,2018-05-30,2018-09-19,2019-01-22,NaT,NaT,2020-02-10,2020-05-12,NaN,None
2,16INR0085,Priddy Wind,"SS Completed, FIS Completed, IA",ENGIE North America,tap 345kV 1444 Brown – 3422 Killeen,Mills,NORTH,2026-12-31,WIN,WT,...,Not Required,2021-03-29,2021-04-30,2021-04-30,NaT,NaT,2021-10-13,2021-11-19,NaN,None
3,16INR0104,Big Sampson Wind,"SS Completed, FIS Completed, IA","Big Sampson Wind Project, LLC",tap 345kV 76015 Cedar Canyon - 7053 Noelke,Crockett,WEST,2026-03-31,WIN,WT,...,Not Required,2024-10-31,2024-11-04,2024-11-04,NaT,NaT,2025-06-04,2025-06-16,NaN,None
5,17INR0027b,Coyote Wind,"SS Completed, FIS Completed, IA",Coyote Wind,11305 Dermott 345kV,Scurry,WEST,2025-12-01,WIN,WT,...,Not Required,2016-08-09,2020-01-28,2020-01-31,NaT,NaT,2020-08-24,2020-11-25,NaN,None
7,17INR0052,Horse13 CallD repower,"SS Completed, FIS Completed, IA",Nextera,6216 Bluff Creek 138kV,Taylor,WEST,2026-09-30,WIN,WT,...,Not Required,2020-04-13,2020-04-13,2020-04-13,NaT,NaT,NaT,2021-03-16,NaN,"(39.5575475, -86.2208223)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,25INR0672,Fagus Solar Park 2 SLF,"SS Completed, FIS Completed, IA","Excel Advantage Services, LLC dba Misae Solar ...",60501 Tesla 345 kV,Childress,PANHANDLE,2026-03-15,SOL,PV,...,Not Required,2024-12-19,2024-12-19,2024-04-30,NaT,NaT,2025-05-29,2025-07-07,NaN,None
857,25INR0673,PHOTON SOLAR 4 SLF,"SS Completed, FIS Completed, No IA","GulfStar Power, LLC",44880 Waterh_POI_5 345kV,Wharton,SOUTH,2026-02-16,SOL,PV,...,Not Required,2021-11-05,2023-01-27,2023-01-27,NaT,NaT,2024-06-25,2024-11-04,NaN,None
1002,26INR0333,VERTUS ENERGY STORAGE,"SS Completed, FIS Completed, IA",VERTUS ENERGY STORAGE LLC,"Bus Name: TnMainland1,\nBus Number: 38830,\n\n",Galveston,HOUSTON,2026-04-28,OTH,BA,...,NaN,2025-02-14,2025-04-16,2025-07-16,NaT,NaT,2025-12-16,2026-01-30,NaN,None
1150,26INR0669,Greyhound Solar - Phase E,"SS Completed, FIS Completed, IA",ROCKHOUND SOLAR 2 LLC DBA GREYHOUND SOLAR,Tap 345kV 11010 Wolf - 1018 Moss,Ector,WEST,2025-11-20,SOL,PV,...,Not Required,2024-11-04,2024-11-04,2024-11-04,NaT,NaT,2025-08-14,2025-09-10,NaN,None


In [27]:
address_to_coordinates('6546 Barnhurst Drive')

(32.8226519, -117.1698717)

In [17]:
import webbrowser

lat, lon = 29.7604, -95.3698
url = f"https://www.google.com/maps/@?api=1&map_action=pano&viewpoint={lat},{lon}"
webbrowser.open(url)

True

In [34]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [35]:
url = 'https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/US-counties.geojson'
gdf_geometries = gpd.read_file(url)

In [38]:
in_play.head()

,INR,Project Name,GIM Study Phase,Interconnecting Entity,POI Location,County,CDR Reporting Zone,Projected COD,Fuel,Technology,...,Water Availability,Meets Planning Guide Section 6.9(1) Requirements for Inclusion in Planning Models,Meets All Planning Guide Section 6.9 Requirements for Inclusion in Planning Models,Meets Planning Guide QSA (Section 5.9) Prerequisites,Construction Start,Construction End,Approved for Energization,Approved for Synchronization,Comment,coordinates
0,15INR0064b,Harald (BearKat Wind B),"SS Completed, FIS Completed, IA","McCrae Wind Energy II, LLC",59903 Bearkat 345kV,Glasscock,WEST,2026-02-28,WIN,WT,...,Not Required,2018-05-30,2018-09-19,2019-01-22,NaT,NaT,2020-02-10,2020-05-12,NaN,None
2,16INR0085,Priddy Wind,"SS Completed, FIS Completed, IA",ENGIE North America,tap 345kV 1444 Brown – 3422 Killeen,Mills,NORTH,2026-12-31,WIN,WT,...,Not Required,2021-03-29,2021-04-30,2021-04-30,NaT,NaT,2021-10-13,2021-11-19,NaN,None
3,16INR0104,Big Sampson Wind,"SS Completed, FIS Completed, IA","Big Sampson Wind Project, LLC",tap 345kV 76015 Cedar Canyon - 7053 Noelke,Crockett,WEST,2026-03-31,WIN,WT,...,Not Required,2024-10-31,2024-11-04,2024-11-04,NaT,NaT,2025-06-04,2025-06-16,NaN,None
5,17INR0027b,Coyote Wind,"SS Completed, FIS Completed, IA",Coyote Wind,11305 Dermott 345kV,Scurry,WEST,2025-12-01,WIN,WT,...,Not Required,2016-08-09,2020-01-28,2020-01-31,NaT,NaT,2020-08-24,2020-11-25,NaN,None
7,17INR0052,Horse13 CallD repower,"SS Completed, FIS Completed, IA",Nextera,6216 Bluff Creek 138kV,Taylor,WEST,2026-09-30,WIN,WT,...,Not Required,2020-04-13,2020-04-13,2020-04-13,NaT,NaT,NaT,2021-03-16,NaN,"(39.5575475, -86.2208223)"


In [36]:
# Ensure the identifier column has the correct data type (e.g., integer for FIPS codes)
gdf_geometries['id'] = gdf_geometries['id'].astype(int)

# 2. Merge your data with the GeoDataFrame (using the 'id' or 'FIPS' column)
# Assume 'df_data' has a 'County_FIPS' and 'Data_Value' column
merged_gdf = pd.merge(in_play, gdf_geometries, left_on='County', right_on='id')

# 3. Plot the choropleth map
merged_gdf.plot(column='Data_Value', cmap='viridis', legend=True, figsize=(10, 6))
plt.title('Map of Data Value by County')
plt.show()

ValueError: You are trying to merge on object and int64 columns for key 'County'. If you wish to proceed you should use pd.concat

In [74]:
import plotly.express as px
import pandas as pd
import json

import geopandas as gpd
import plotly.express as px
import json

# 1. Load shapefile
gdf = gpd.read_file("/home/dell/code/ercot/maps/tl_2025_us_county/tl_2025_us_county.shp")

# 2. Filter Texas only (STATEFP = 48)
gdf_tx = gdf[gdf["STATEFP"] == "48"]

# 3. Convert to GeoJSON dict
counties_geojson = json.loads(gdf_tx.to_json())


In [76]:
import re

def norm_county(s: str) -> str:
    if s is None:
        return ""
    s = str(s).upper().strip()
    s = re.sub(r"\s+COUNTY$", "", s)      # remove trailing " COUNTY"
    s = re.sub(r"[^\w\s-]", "", s)        # drop punctuation
    s = re.sub(r"\s+", " ", s)            # collapse spaces
    return s

in_play["County_clean"] = in_play["County"].map(norm_county)
gdf_tx["NAME_clean"] = gdf_tx["NAME"].map(norm_county)

merged = gdf_tx.merge(
    in_play,
    left_on="NAME_clean",
    right_on="County_clean",
    how="left"
)


/home/dell/miniconda3/envs/stock/lib/python3.12/site-packages/geopandas/geodataframe.py:1969: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [81]:
import numpy as np

merged["Capacity_plot"] = merged["Capacity (MW)"]
merged.loc[merged["Capacity_plot"] == 0, "Capacity_plot"] = np.nan

fig = px.choropleth(
    merged,
    geojson=json.loads(merged.to_json()),
    locations="GEOID",
    featureidkey="properties.GEOID",
    color="Capacity_plot",
    color_continuous_scale="Viridis",
    hover_name="NAME",
    scope="usa",
    title="Texas Capacity by County"
)

fig.update_geos(fitbounds="locations", visible=False)
fig.show()


TypeError: Object of type Timestamp is not JSON serializable

In [72]:
fig.write_html(out_html, include_plotlyjs="cdn", full_html=True)
out_html = "/home/dell/code/ercot/tx_capacity_map_new.html"
print("Saved:", out_html)

Saved: /home/dell/code/ercot/tx_capacity_map_new.html


In [ ]:
import json
import numpy as np
import pandas as pd
import plotly.express as px

# Make plot column
merged["Capacity_plot"] = merged["Capacity (MW)"]
merged.loc[merged["Capacity_plot"] == 0, "Capacity_plot"] = np.nan

# Slim down to avoid Timestamp serialization issues
geo_cols = ["GEOID", "NAME", "Capacity_plot", "geometry"]
g_geo = merged[geo_cols].copy()

# Ensure CRS is lat/lon for Plotly
# (only needed once; harmless if already 4326)
try:
    g_geo = g_geo.to_crs(epsg=4326)
except Exception:
    pass

counties_geojson = json.loads(g_geo.to_json())

fig = px.choropleth(
    g_geo,
    geojson=counties_geojson,
    locations="GEOID",
    featureidkey="properties.GEOID",
    color="Capacity_plot",
    color_continuous_scale="Viridis",
    hover_name="NAME",
    scope="usa",
    title="Texas Capacity by County"
)
fig.update_geos(fitbounds="locations", visible=False)

# show in notebook
fig.show()

# save html
out_path = "/home/dell/code/ercot/maps/tx_capacity_map.html"
fig.write_html(out_path, include_plotlyjs="cdn", full_html=True)
print("Saved:", out_path)
